## SERP 데이터

In [1]:
# http://ciir.cs.umass.edu/downloads/mimics-serp/MIMICS-BingAPI-results.zip
f = open("MIMICS-BingAPI.sample")
dataset = f.readlines()
f.close()

In [182]:
len(dataset)

10000

In [196]:
data = dataset[1]
data = data.replace("true", "True")
data = data.replace("false", "False")

In [197]:
json_data = eval(data)
json_data.keys()

dict_keys(['instrumentation', 'queryContext', 'webPages', 'relatedSearches', 'videos', 'rankingResponse', 'pagination'])

In [198]:
json_data['instrumentation']

{'_type': 'ResponseInstrumentation',
 'pingUrlBase': 'https://www.bingapis.com/api/ping?IG=698A77DC18C34087B3411EE209FAEDE8&CID=3EBE2CEC73FF60FA0B8723F972B3613A&ID=',
 'pageLoadPingUrl': 'https://www.bingapis.com/api/ping/pageload?IG=698A77DC18C34087B3411EE209FAEDE8&CID=3EBE2CEC73FF60FA0B8723F972B3613A&Type=Event.CPT&DATA=0'}

In [199]:
query = json_data['queryContext']['originalQuery']
query

'aveda products'

In [200]:
json_data['webPages'].keys()

dict_keys(['webSearchUrl', 'webSearchUrlPingSuffix', 'totalEstimatedMatches', 'value'])

In [201]:
snippet_list = [x['snippet'] for x in json_data['webPages']['value']]
snippet_list

['Discover <b>Aveda</b>&#39;s natural, eco-friendly <b>products</b> designed to make you look &amp; feel fresh. Shop professional hair &amp; skin care <b>products</b> or find a salon near you!',
 '<b>Aveda</b> Styling <b>Products</b> $. $. Pure Abundance Hair Potion By <b>Aveda</b> - 0.7 Oz Potion <b>Aveda</b> Light Elements Shaping Wax <b>Aveda</b> Air Control Hairspray Travel Size 1.5 Oz <b>Aveda</b> Phomollient Styling Foam 33.8 oz <b>Aveda</b> Brilliant Medium Hold Hair Spray 33.8 Oz <b>Aveda</b> Men Pure-formance ...',
 'Amazon&#39;s Choice for <b>aveda products</b>. <b>Aveda</b> Volumizing Tonic with Aloe, 3.4oz. 4.5 out of 5 stars 365. $23.09 $ 23. 09 ($6.79/Fl Oz) Get it as soon as Mon, Jul 27. FREE Shipping on your first order shipped by Amazon. Only 20 left in stock - order soon. More Buying Choices $22.50 (19 new offers)',
 'Whichever <b>products</b> you choose, you can count on <b>Aveda</b> to deliver planet-friendly <b>products</b> featuring naturally derived ingredients a

In [202]:
json_data['webPages']['value'][0]['id'], json_data['webPages']['value'][0]['url']

('https://www.bingapis.com/api/v7/#WebPages.0', 'https://www.aveda.com/')

In [203]:
json_data['relatedSearches'].keys()

dict_keys(['id', 'value'])

In [204]:
json_data['relatedSearches']['id']

'https://www.bingapis.com/api/v7/#RelatedSearches'

In [205]:
json_data['relatedSearches']['value'][0]

{'text': 'aveda hair products',
 'displayText': 'aveda <b>hair</b> products',
 'webSearchUrl': 'https://www.bing.com/search?q=aveda+hair+products',
 'webSearchUrlPingSuffix': 'DevEx,5516.1'}

In [206]:
related_list = [x['text'] for x in json_data['relatedSearches']['value']]
related_list

['aveda hair products',
 'nordstrom aveda products',
 'aveda products for thinning hair',
 'aveda products where to buy',
 'aveda products for gray hair',
 'aveda hair products sale',
 'aveda hair products for women',
 'aveda hair products at target']

In [207]:
json_data['rankingResponse'].keys()

dict_keys(['mainline', 'pagination'])

In [208]:
json_data['rankingResponse']['mainline']['items'][0]['value']['id']

'https://www.bingapis.com/api/v7/#WebPages.0'

In [209]:
# json_data

import jsonlines

with jsonlines.open("target.jsonl") as f:
    for line in f.iter():
    	print(line["id"]) # 각 json에 해당하는 "id" 출력
        print(line["title"]) # 각 json에 해당하는 "title" 출력### SERP queries

In [1]:
import jsonlines

dataset = []
with jsonlines.open("MIMICS-BingAPI.jsonl") as f:
    for line in f.iter():
        dataset.append(line)

In [3]:
len(dataset), dataset[0].keys()

(479807, dict_keys(['what is superfetch']))

In [5]:
query_document = {}
for data in dataset:
    query = list(data.keys())[0]
    snippet_list = data[query]
    query_document[query] = snippet_list

In [6]:
len(query_document)

479807

## MIMICS 데이터

In [7]:
import glob
folder = "MIMICS/data/*"
datapath_list = glob.glob(folder)
datapath_list

['MIMICS/data/MIMICS-Click.tsv',
 'MIMICS/data/MIMICS-Manual.tsv',
 'MIMICS/data/MIMICS-ClickExplore.tsv']

In [8]:
train_path = datapath_list[0]
test_path = datapath_list[1]

In [9]:
import pandas as pd
tr_df = pd.read_csv(train_path, sep = '\t')
te_df = pd.read_csv(test_path, sep = '\t')

In [10]:
type(tr_df)

pandas.core.frame.DataFrame

In [11]:
tr_df.shape

(414362, 14)

In [12]:
tr_df.columns

Index(['query', 'question', 'option_1', 'option_2', 'option_3', 'option_4',
       'option_5', 'impression_level', 'engagement_level', 'option_cctr_1',
       'option_cctr_2', 'option_cctr_3', 'option_cctr_4', 'option_cctr_5'],
      dtype='object')

In [13]:
train_dataset = tr_df[['query', 'option_1', 'option_2', 'option_3', 'option_4', 'option_5']] # Click
test_dataset = te_df[['query', 'option_1', 'option_2', 'option_3', 'option_4', 'option_5', 'options_overall_label']] # Manual

In [26]:
import math
train_data = {}
for i in range(len(train_dataset)):
    data = train_dataset.iloc[i]
    query, option_1, option_2, option_3, option_4, option_5 = data['query'], data['option_1'], data['option_2'], data['option_3'], data['option_4'], data['option_5']    
    train_data[i] = {}
    train_data[i]['query'] = query
    options = [option_1, option_2, option_3, option_4, option_5]
    filter_options = []
    for option in options:
        if not (isinstance(option, float) and (math.isnan(option))):
            filter_options.append(option)    
    train_data[i]['facet'] = filter_options
    if query in query_document:
        train_data[i]['document'] = query_document[query]
    
test_data = {}
for i in range(len(test_dataset)):
    data = test_dataset.iloc[i]
    query, option_1, option_2, option_3, option_4, option_5 = data['query'], data['option_1'], data['option_2'], data['option_3'], data['option_4'], data['option_5']    
    test_data[i] = {}
    test_data[i]['query'] = query
    options = [option_1, option_2, option_3, option_4, option_5]
    filter_options = []
    for option in options:
        if not (isinstance(option, float) and (math.isnan(option))):
            filter_options.append(option)
    test_data[i]['facet'] = filter_options
    
    options_overall_label = int(data['options_overall_label']) # Manual
    test_data[i]['options_overall_label'] = options_overall_label
    if query in query_document:
        test_data[i]['document'] = query_document[query]

In [15]:
print(len(train_data), len(test_data))

414362 2832


In [16]:
docu_num = 0
for k, data in train_data.items():
    if 'document' in data:
        docu_num += 1
print(docu_num)        

414202


In [17]:
docu_num = 0
for k, data in test_data.items():
    if 'document' in data:
        docu_num += 1
print(docu_num)        

2829


In [38]:
from tqdm import tqdm
train_docu_data = []
num = 0
for i, data in tqdm(train_data.items()):
    query, facet = data['query'], data['facet']
    if "atropine sulfate drops" in query:
        print(query)
        print(facet)
        num+=1    
    if "best fps games" in query:
        print(query)
        print(facet)
        num+=1    
    if "accident investigation" in query:
        print(query)
        print(facet)
        num+=1
    if num == 3:
        break

 10%|█         | 42018/414362 [00:00<00:00, 1514343.48it/s]

accident investigation
['motorcycle accident', 'car accident', 'train accident', 'boat accident', 'forklift accident']
atropine sulfate drops
['interactions', 'overdose', 'precautions', 'side effects', 'uses']
best fps games
['for steam', 'for pc', 'for ps4', 'for xbox one', 'for android']


In [10]:
import json
with open("data/train.json", 'w', encoding='utf-8') as f:
    json.dump(train_data, f, indent="\t")

In [28]:
with open("data/test.json", 'w', encoding='utf-8') as f:
    json.dump(test_data, f, indent="\t")

In [11]:
import json
with open("data/train.json", 'r', encoding='utf-8') as f:
    train_data = json.load(f)

In [13]:
for k, v in train_data.items():
    if 'document' not in v:
        print(v)
        break

{'query': 'acronis true image', 'facet': ['windows 10', 'windows xp', 'windows 7', 'mac os x', 'windows 2000']}


In [14]:
v['document']

KeyError: 'document'